# PCA on traits

In [1]:
import os
import sys
import torch
import pandas as pd

sys.path.append('.')
sys.path.append('..')

from utils.pca_utils import *
from plots import *

## Configuration

In [2]:
# Configuration - Change these parameters for different models/datasets
base_dir = "/workspace/llama-3.3-70b"
type = "traits_240"
dir = f"{base_dir}/{type}"
model_name = "Llama-3.3-70B"
layer = 40

## Load vectors

In [3]:
# load all vectors from vectors
vector_dir = f"{dir}/vectors"

# iterate through each .pt file in the directory
vectors = {}
for file in os.listdir(vector_dir):
    if file.endswith(".pt"):
        vectors[file.replace(".pt", "")] = torch.load(os.path.join(vector_dir, file))

print(f"Found {len(vectors.keys())} traits with vectors")

Found 240 traits with vectors


In [4]:
# load default vectors
default_vectors = torch.load(f"{base_dir}/roles_240/default_vectors.pt")

In [5]:
print(vectors['zealous'].keys())
print(default_vectors.keys())
print(default_vectors['activations'].keys())

dict_keys(['pos_neg', 'pos_neg_50', 'pos_default', 'pos_default_50', 'pos_70', 'pos_40_70'])
dict_keys(['activations', 'metadata'])
dict_keys(['pos_1', 'default_1', 'all_1'])


## PCA

In [8]:
# load in stats
results_subdir = model_name.lower()
stats = pd.read_csv(f"./results/{results_subdir}/{type}/pos_neg.csv", index_col='trait')
print(stats.loc['zealous']['large_diff_count'])

1181.0


In [9]:
# PCA on pos_neg_50 but filter out traits with large_diff_count < 10
filtered_pos_neg_50_traits = []
filtered_pos_neg_50 = []

for trait, vector in vectors.items():
    if stats.loc[trait]['large_diff_count'] >= 10:
        filtered_pos_neg_50_traits.append(trait)
        filtered_pos_neg_50.append(vector['pos_neg_50'])
    else:
        print(f"Skipping {trait} because large_diff_count is {stats.loc[trait]['large_diff_count']}")

print(len(filtered_pos_neg_50_traits))

NameError: name 'vectors' is not defined

In [45]:
pca_transformed, variance_explained, n_components, pca, scaler = compute_pca(torch.stack(filtered_pos_neg_50).float(), layer)

PCA fitted with 240 components
Cumulative variance for first 5 components: [0.23095817 0.37114492 0.47023114 0.54977089 0.60345525]

PCA Analysis Results:
Elbow point at component: 2
Dimensions for 70% variance: 8
Dimensions for 80% variance: 14
Dimensions for 90% variance: 31
Dimensions for 95% variance: 55


In [46]:
# save pca
results = {}

results['layer'] = layer
results['traits'] = {
    'pos_neg_50': filtered_pos_neg_50_traits
}
results['vectors'] = {
    'pos_neg_50': filtered_pos_neg_50,
}
results['pca_transformed'] = pca_transformed
results['variance_explained'] = variance_explained
results['n_components'] = n_components
results['pca'] = pca
results['scaler'] = scaler

pca_dir = f"{dir}/pca"
os.makedirs(pca_dir, exist_ok=True)
torch.save(results, f"{pca_dir}/layer{layer}_pos-neg50.pt")

## Plots

In [3]:
plot_dir = f"./results/{model_name.lower()}/pca_240"
os.makedirs(plot_dir, exist_ok=True)

In [4]:
default_vectors = torch.load(f"{base_dir}/roles_240/default_vectors.pt")
pca_results = torch.load(f"{dir}/pca/layer{layer}_pos-neg50.pt", weights_only=False)

In [5]:
# get default activationa nd project into PCA space
assistant_layer_activation = default_vectors['activations']['default_1'][layer, :].float().numpy().reshape(1, -1)
asst_scaled = pca_results['scaler'].transform(assistant_layer_activation)
asst_projected = pca_results['pca'].transform(asst_scaled)

In [6]:
trait_labels = [trait.replace('_', ' ').capitalize() for trait in pca_results['traits']['pos_neg_50']]

In [11]:
for i in range(10):
    fig = plot_pc(
        pca_results=pca_results,
        trait_labels=trait_labels,
        layer=layer,
        pc_component=i,
        assistant_activation=default_vectors['activations']['default_1'],
        assistant_projection=asst_projected[0],
        subtitle=f"{model_name.replace('-', ' ')}, Layer {layer} - Shared Question Set"
    )
    fig.show()
    fig.write_html(f"{plot_dir}/pc{i+1}.html")

In [8]:
# for i in range(10):
#     fig = plot_pca_cosine_similarity(
#             pca_results=pca_results,
#             trait_labels=trait_labels,
#             pc_component=i,
#             layer=layer,
#             assistant_activation=default_vectors['activations']['default_1'],
#             subtitle=f"{model_name.replace('-', ' ')}, Layer {layer} - Shared Question Set"
#         )
#     fig.show()
#     fig.write_html(f"{plot_dir}/pc{i+1}_cossim.html")

In [9]:
# for i in range(10):
#     fig = plot_pca_projection(
#             pca_results=pca_results,
#             trait_labels=trait_labels,
#             pc_component=i,
#             assistant_activation=asst_projected[0],
#             subtitle=f"{model_name.replace('-', ' ')}, Layer {layer} - Shared Question Set"
#         )
#     fig.show()
#     fig.write_html(f"{plot_dir}/pc{i+1}_projection.html")

In [10]:
fig_3d = plot_3d_pca(
    pca_results['pca_transformed'],
    pca_results['variance_explained'],
    trait_labels,
    assistant_projection=asst_projected[0],
    title="Trait Vectors in 3D Principal Component Space",
    subtitle=f"{model_name.replace('-', ' ')}, Layer {layer} - Shared Question Set"
)
fig_3d.show()
fig_3d.write_html(f"{plot_dir}/pca_3d.html")